In [ ]:
from IPython.display import clear_output

!git clone https://github.com/deepmind/clrs

%cd clrs
!pip install -r requirements/requirements.txt
#!pip install dm-clrs
#for assert
!pip install chex

clear_output(wait=False)

In [ ]:
import clrs
import importlib
importlib.reload(clrs)
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
import importlib
importlib.reload(clrs)
from jax import random
from jax import grad

import haiku as hk
import jax
import jax.numpy as jnp
import optax
import numpy as np
import pandas as pd
import plotnine as gg

batch_size = 1
hidden_dim = 3
nb_nodes = 3


def loss_fn(data):
  mlp = hk.Sequential([
      clrs._src.processors.GATv2(10, 1)
  ])


  logits = mlp(data[0], data[1], data[2], data[3], data[4])
  return jnp.mean(logits)

loss_fn_t = hk.transform(loss_fn)

node_fts = random.uniform(random.PRNGKey(0), shape = (batch_size, nb_nodes, hidden_dim))#jnp.zeros((batch_size, nb_nodes, hidden_dim))
edge_fts = random.uniform(random.PRNGKey(0), shape = (batch_size, nb_nodes, nb_nodes, hidden_dim))#jnp.zeros((batch_size, nb_nodes, nb_nodes, hidden_dim))
graph_fts = random.uniform(random.PRNGKey(0), shape = (batch_size, hidden_dim))#jnp.zeros((batch_size, hidden_dim))
adj_mat = random.randint(random.PRNGKey(0), (batch_size, nb_nodes, nb_nodes), 0, 2, dtype='uint8')
hiddens = jnp.zeros((batch_size, nb_nodes, hidden_dim))

rng = jax.random.PRNGKey(42)
data = (node_fts, edge_fts, graph_fts, adj_mat, hiddens)
params = loss_fn_t.init(rng, data)


def loss(params, data):
  rng = jax.random.PRNGKey(42)
  return loss_fn_t.apply(params, rng, data)

def update(params, data, opt_state):
  step_size = 0.1
  rng = jax.random.PRNGKey(42)
  l, grads = jax.value_and_grad(loss)(params, data)
  grads, opt_state = opt.update(grads, opt_state)
  params = optax.apply_updates(params, grads)
  return l, params, opt_state

opt = optax.adam(1e-3)
opt_state = opt.init(params)


train_loss, params, opt_state = update(params, data, opt_state)


In [ ]:
params.keys()

dict_keys(['gatv2_aggr/linear', 'gatv2_aggr/linear_1', 'gatv2_aggr/linear_2', 'gatv2_aggr/linear_3', 'gatv2_aggr/linear_4', 'gatv2_aggr/linear_5', 'gatv2_aggr/linear_6'])